In [1]:
from glob import glob
from shutil import copyfile
import os

import numpy as np
import pandas as pd

from keras.layers.core import Dense, Dropout
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers.core import *

from fastai.vgg16bn import Vgg16BN
from fastai.utils import *

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


# Setup Files

In [7]:
%cd statefarm
# %mkdir valid
# %mkdir results
# %mkdir -p sample/train
# %mkdir -p sample/test
# %mkdir -p sample/valid
# %mkdir -p sample/results

/home/ubuntu/projects/statefarm


In [12]:
%cd train

/home/ubuntu/projects/statefarm/train


In [13]:
g = glob('*')
for d in g: 
#     os.mkdir('../sample/train/'+d)
#     os.mkdir('../sample/valid/'+d)
    os.mkdir('../valid/'+d)
%ls ../sample/train

c0/  c1/  c2/  c3/  c4/  c5/  c6/  c7/  c8/  c9/


In [ ]:
g = glob('*/*.jpg')
sample_size = int(len(g) * .25)
shuf = np.random.permutation(g)
for i in range(sample_size): copyfile(shuf[i], '../sample/train/' + shuf[i])

In [ ]:
g = glob('*/*.jpg')
valid_size = int(len(g) * .1)
shuf = np.random.permutation(g)
for i in range(valid_size): copyfile(shuf[i], '../sample/valid/' + shuf[i])

In [14]:
%cd ../..

/home/ubuntu/projects


In [15]:
%cd statefarm

/home/ubuntu/projects/statefarm


Select a couple of subjects for validation

In [16]:
imgs = pd.read_csv('driver_imgs_list.csv')
print imgs.head()
subjects = imgs['subject'].unique()
valid_set = imgs[imgs['subject'].isin(subjects[:3])]
print valid_set.shape

def copy_img(r):
    p = os.path.join(r['classname'], r['img'])
    os.rename('train/' + p, 'valid/' + p)

valid_set.apply(copy_img, axis=1)

  subject classname            img
0    p002        c0  img_44733.jpg
1    p002        c0  img_72999.jpg
2    p002        c0  img_25094.jpg
3    p002        c0  img_69092.jpg
4    p002        c0  img_92629.jpg
(2424, 3)


0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
2394    None
2395    None
2396    None
2397    None
2398    None
2399    None
2400    None
2401    None
2402    None
2403    None
2404    None
2405    None
2406    None
2407    None
2408    None
2409    None
2410    None
2411    None
2412    None
2413    None
2414    None
2415    None
2416    None
2417    None
2418    None
2419    None
2420    None
2421    None
2422    None
2423    None
dtype: object

In [17]:
g = glob('valid/*/*.jpg')
len(g)

2424

In [18]:
%cd ..

/home/ubuntu/projects


# Load Images

In [4]:
sample_path = 'statefarm/sample/'
full_path = 'statefarm/'
path = full_path

In [19]:
gen = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)

train = get_batches(path+'train', gen, shuffle=False)
valid = get_batches(path+'valid', shuffle=False)
train_labels = onehot(train.classes)
valid_labels = onehot(valid.classes)

Found 20000 images belonging to 10 classes.
Found 2424 images belonging to 10 classes.


# Precompute Conv Outputs

In [2]:
vgg_conv = Vgg16BN(include_top=False)
keras.__version__

'1.1.0'

In [20]:
sets = 5

for i in range(sets):
    train_conv_feat = vgg_conv.model.predict_generator(train, train.nb_sample)
    save_array(full_path + 'train_conv_feat{}.dat'.format(i+1), train_conv_feat)
    print('.')

.
.
.
.
.


In [21]:
valid_conv_feat = vgg_conv.model.predict_generator(valid, valid.nb_sample)

In [22]:
save_array(full_path + 'valid_conv_feat.dat', valid_conv_feat)

In [4]:
valid_conv_feat = load_array(full_path + 'valid_conv_feat.dat')

# Training

In [9]:
p = 0.8
dense_layers = [
#         MaxPooling2D(),
        Flatten(input_shape=vgg_conv.model.layers[-1].output_shape[1:]),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]
vgg_den = Sequential(dense_layers)
vgg_den.summary()
vgg_den.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
flatten_1 (Flatten)              (None, 25088)         0           flatten_input_1[0][0]            
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 25088)         0           flatten_1[0][0]                  
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 256)           6422784     dropout_1[0][0]                  
____________________________________________________________________________________________________
batchnormalization_1 (BatchNormal(None, 256)           512         dense_1[0][0]                    
___________________________________________________________________________________________

In [25]:
sets = 5

for i in range(sets):
    train_conv_feat = load_array(full_path + 'train_conv_feat{}.dat'.format(i+1))
    vgg_den.fit(train_conv_feat, train_labels, batch_size=64, nb_epoch=5, 
                 validation_data=(valid_conv_feat, valid_labels), verbose=1)

Train on 20000 samples, validate on 2424 samples
Epoch 1/5
20000/20000 [==============================] - 5s - loss: 0.4963 - acc: 0.8424 - val_loss: 0.7714 - val_acc: 0.7450
Epoch 2/5
20000/20000 [==============================] - 5s - loss: 0.4809 - acc: 0.8485 - val_loss: 0.7303 - val_acc: 0.7599
Epoch 3/5
20000/20000 [==============================] - 5s - loss: 0.4584 - acc: 0.8558 - val_loss: 0.8036 - val_acc: 0.7360
Epoch 4/5
20000/20000 [==============================] - 4s - loss: 0.4374 - acc: 0.8601 - val_loss: 0.7988 - val_acc: 0.7261
Epoch 5/5
20000/20000 [==============================] - 4s - loss: 0.4289 - acc: 0.8650 - val_loss: 0.7516 - val_acc: 0.7483
Train on 20000 samples, validate on 2424 samples
Epoch 1/5
20000/20000 [==============================] - 4s - loss: 0.5680 - acc: 0.8363 - val_loss: 0.8201 - val_acc: 0.7079
Epoch 2/5
20000/20000 [==============================] - 4s - loss: 0.5390 - acc: 0.8442 - val_loss: 0.8196 - val_acc: 0.7285
Epoch 3/5
20000/2000

In [26]:
vgg_den.save_weights(full_path + 'statefarm.dense.1.h5')

In [10]:
vgg_den.load_weights(full_path + 'statefarm.dense.1.h5')

# Predict

In [5]:
batches, preds = vgg_conv.test(path + 'test', batch_size=128)
print path + 'test'
print batches.filenames[:5]
print preds[:5]

Found 79726 images belonging to 1 classes.
statefarm/test
['cx/img_42265.jpg', 'cx/img_56819.jpg', 'cx/img_44646.jpg', 'cx/img_37236.jpg', 'cx/img_43344.jpg']
[[[[   0.        2.1658    0.     ...,    0.        0.        0.    ]
   [   0.        0.        0.     ...,    0.        0.        0.    ]
   [   0.        0.        0.     ...,    0.        0.        0.    ]
   ..., 
   [   0.        0.        0.     ...,    3.2289   23.9373    0.    ]
   [   0.        0.        5.0505 ...,   34.5178   39.806     9.0423]
   [   2.1061    0.        0.     ...,    0.        0.       14.2771]]

  [[  46.5832   39.3232    0.     ...,    0.        0.        0.    ]
   [   0.        0.        0.     ...,   31.3896   27.774     0.    ]
   [   6.0242    0.        0.     ...,   88.6597   92.3857    1.9603]
   ..., 
   [  16.1446    0.        0.     ...,    0.        0.        0.    ]
   [   0.        0.        0.     ...,    0.        0.        0.    ]
   [   0.        0.        0.     ...,    0.       

In [7]:
save_array(full_path + 'test_conv_preds.dat', preds)

In [11]:
preds = vgg_den.predict(preds, verbose=1)

79712/79726 [============================>.] - ETA: 0s

In [12]:
save_array(full_path + 'preds.dat', preds)
save_array(full_path + 'filenames.dat', batches.filenames)

In [13]:
import pandas as pd
#subm = np.hstack((np.array(batches.filenames), np.array(preds)))
filenames = [ os.path.basename(f) for f in batches.filenames ]
subm = pd.DataFrame(preds, index=filenames, columns=['c0','c1','c2','c3','c4','c5','c6','c7','c8','c9'])
subm.index.name = 'img'
subm.to_csv(full_path + 'submission1.csv')

In [14]:
from IPython.display import FileLink
FileLink(full_path + 'submission1.csv')

/home/ubuntu/projects/statefarm/submission1.csv

**Scores:**
2.26947, 2.75651
0.61808, 0.67364